# K-Means UDTF Sample Usage

This notebook demonstrates how to use the k-means UDTF with the Titanic dataset.

In [1]:
# Create sample Titanic data
spark.sql("""
    CREATE OR REPLACE TABLE main.test.titanic_sample (
        PassengerId INT,
        Age DOUBLE,
        Pclass INT,
        Survived INT
    )
""")

spark.sql("""
    INSERT INTO main.test.titanic_sample VALUES
        (1, 22.0, 3, 0),
        (2, 38.0, 1, 1),
        (3, 26.0, 3, 1),
        (4, 35.0, 1, 1),
        (5, 35.0, 3, 0),
        (6, 54.0, 1, 0),
        (7, 2.0, 3, 0),
        (8, 27.0, 1, 0),
        (9, 14.0, 3, 1),
        (10, 4.0, 3, 1)
""")

,num_affected_rows,num_inserted_rows
0,10,10


In [ ]:
# Use k-means UDTF to cluster passengers
# Assumes the UDTF has been deployed to main.${user}.k_means

spark.sql("""
    WITH kmeans AS (
        SELECT *
        FROM main.lennart.k_means(
            input_data => TABLE(SELECT * FROM main.test.titanic_sample),
            id_column => 'PassengerId',
            columns => array('Age', 'Pclass', 'Survived'),
            k => 3
        )
    )
    SELECT 
        t.*,
        k.cluster_id
    FROM main.test.titanic_sample t
    JOIN kmeans k ON CAST(t.PassengerId AS STRING) = k.id
    ORDER BY t.PassengerId
""").display()

In [ ]:
# Analyze cluster characteristics
spark.sql("""
    WITH kmeans AS (
        SELECT *
        FROM main.lennart.k_means(
            input_data => TABLE(SELECT * FROM main.test.titanic_sample),
            id_column => 'PassengerId',
            columns => array('Age', 'Pclass', 'Survived'),
            k => 3
        )
    )
    SELECT 
        k.cluster_id,
        COUNT(*) as count,
        AVG(t.Age) as avg_age,
        AVG(t.Pclass) as avg_class,
        AVG(t.Survived) as survival_rate
    FROM main.test.titanic_sample t
    JOIN kmeans k ON CAST(t.PassengerId AS STRING) = k.id
    GROUP BY k.cluster_id
    ORDER BY k.cluster_id
""").display()